In [13]:
def train(data):
    sentences = data.split('\n') #Разделим нашу строку на предложения
    tokenized_corpus = []
    import re
    for sentence in sentences:
        reg = re.compile('[^a-zA-Z ]')
        sentence = reg.sub('', sentence)
        tokenized_sentense = sentence.split()
        tokenized_corpus.append(tokenized_sentense)
    # На этом этапе в tokenized_corpus лежит массив предложений, которые представляют собой массив слов
    
    vocabulary = {} 
    index = 0
    for sentense in tokenized_corpus:
        for token in sentense:
            if token not in vocabulary:
                vocabulary[token] = index
                index += 1
    vocabulary_size = len(vocabulary)
    #vocabulary - словарь вида (Слово - индекс). Нам понадобится словарь вида (индекс - слово) - ind_word
    ind_word = {v:k for k, v in vocabulary.items()}
    
    window_size = 2
    pairs = []
    
    #Создадим пары слов (центральное слово - контекстное слово)
    for sentense in tokenized_corpus:
        indices = []
        for word in sentense:
            indices.append(vocabulary[word])
        
        for mid_pos in range(len(indices)):
            for stride in range(-window_size, window_size + 1):
                    context_pos = mid_pos + stride
                
                    if context_pos < 0 or context_pos >= len(indices) or context_pos == mid_pos:
                        continue
                    
                    context_index = indices[context_pos]
                
                    pairs.append((indices[mid_pos], context_index))
      # В pairs в итоге лежат индексы пар контекстно-связанных слов
    
  #Создадим функцию, которая возвращает one-hot вектор слова
    def get_input_layer(word_idx):
        x = torch.zeros(vocabulary_size).float()
        x[word_idx] = 1.0
        return x
    
    #Не забудем импортировать необходимые библиотеки

    import numpy as np
    import torch
    from torch.autograd import Variable
    import torch.nn.functional as F
    
    #Делаем training_loop
    embedding_dims = 5
    W1 = Variable(torch.randn(embedding_dims, vocabulary_size).float(), requires_grad=True)
    W2 = Variable(torch.randn(vocabulary_size, embedding_dims).float(), requires_grad=True)
    num_epochs = 100
    learning_rate = 0.001

    for epo in range(num_epochs):
        loss_val = 0
        for data, target in pairs:
            x = Variable(get_input_layer(data)).float()
            y_true = Variable(torch.from_numpy(np.array([target])).long())

            z1 = torch.matmul(W1, x)
            z2 = torch.matmul(W2, z1)
    
            log_softmax = F.log_softmax(z2, dim=0)

            loss = F.nll_loss(log_softmax.view(1,-1), y_true)
            loss_val += loss.data
            loss.backward()
            W1.data -= learning_rate * W1.grad.data
            W2.data -= learning_rate * W2.grad.data

            W1.grad.data.zero_()
            W2.grad.data.zero_()
            
    #Натренировали матрицы весов, теперь делаем словарь вида (слово - векторное представление слова)
    w2v_dict = {}
    for word in vocabulary:
        x = get_input_layer(vocabulary[word]).float()
        z1 = torch.matmul(W1, x)
        word_vector = z1.detach().numpy()
        w2v_dict[word] = word_vector
 
    return w2v_dict






In [14]:
def train(data):
    
    sentences = data.split('\n') #Разделим нашу строку на предложения
    tokenized_corpus = []
    for sentence in sentences:
        tokenized_sentense = sentence.split()
        tokenized_corpus.append(tokenized_sentense)
    # На этом этапе в tokenized_corpus лежит массив предложений, которые представляют собой массив слов
    
    vocabulary = {} 
    index = 0
    for sentense in tokenized_corpus:
        for token in sentense:
            if token not in vocabulary:
                vocabulary[token] = index
                index += 1
    vocabulary_size = len(vocabulary)
    #vocabulary - словарь вида (Слово - индекс). Нам понадобится словарь вида (индекс - слово) - ind_word
    ind_word = {v:k for k, v in vocabulary.items()}
    
    window_size = 2
    pairs = []
    
    #Создадим пары слов (центральное слово - контекстное слово)
    for sentense in tokenized_corpus:
        indices = []
        for word in sentense:
            indices.append(vocabulary[word])
        
        for mid_pos in range(len(indices)):
            for stride in range(-window_size, window_size + 1):
                    context_pos = mid_pos + stride
                
                    if context_pos < 0 or context_pos >= len(indices) or context_pos == mid_pos:
                        continue
                    
                    context_index = indices[context_pos]
                
                    pairs.append((indices[mid_pos], context_index))
      # В pairs в итоге лежат индексы пар контекстно-связанных слов
    
  #Создадим функцию, которая возвращает one-hot вектор слова
    def get_input_layer(word_idx):
        x = torch.zeros(vocabulary_size).float()
        x[word_idx] = 1.0
        return x
    
    #Не забудем импортировать необходимые библиотеки

    import numpy as np
    import torch
    from torch.autograd import Variable
    import torch.nn.functional as F
    
    #Делаем training_loop
    embedding_dims = 50
    W1 = Variable(torch.randn(embedding_dims, vocabulary_size).float(), requires_grad=True)
    W2 = Variable(torch.randn(vocabulary_size, embedding_dims).float(), requires_grad=True)
    num_epochs = 100
    learning_rate = 0.001

    for epo in range(num_epochs):
        loss_val = 0
        for data, target in pairs:
            x = Variable(get_input_layer(data)).float()
            y_true = Variable(torch.from_numpy(np.array([target])).long())

            z1 = torch.matmul(W1, x)
            z2 = torch.matmul(W2, z1)
    
            log_softmax = F.log_softmax(z2, dim=0)

            loss = F.nll_loss(log_softmax.view(1,-1), y_true)
            loss_val += loss.data
            loss.backward()
            W1.data -= learning_rate * W1.grad.data
            W2.data -= learning_rate * W2.grad.data

            W1.grad.data.zero_()
            W2.grad.data.zero_()
            
    #Натренировали матрицы весов, теперь делаем словарь вида (слово - векторное представление слова)
    w2v_dict = {}
    for word in vocabulary:
        x = get_input_layer(vocabulary[word]).float()
        z1 = torch.matmul(W1, x)
        word_vector = z1.detach().numpy()
        w2v_dict[word] = word_vector

    return w2v_dict






In [15]:
corpus = 'for many years I have been\n waiting for a human \n who can beat the dragon \n in this case i will marry\n this breathtaking knight '

In [16]:
train(corpus)

{'for': array([ 1.0304762 ,  1.4326396 , -0.39968315, -1.6045033 ,  1.1524416 ,
        -1.6550056 ,  0.2869772 ,  0.15982555, -0.8191672 , -1.9239312 ,
         2.66039   , -0.05731542,  0.9129042 ,  1.841834  ,  0.11873593,
         0.96814257, -1.9877588 , -0.06712463,  0.09862917,  0.6735367 ,
         0.22900686,  0.6451886 , -2.052291  ,  0.9945709 ,  0.6078292 ,
        -0.23910455, -0.3745055 , -0.72732043, -0.0743685 , -2.3655984 ,
         0.2367261 , -0.55405074, -0.04976449,  0.5208872 , -0.49788198,
        -0.22971793,  0.2196353 , -0.04161566, -1.2319247 ,  0.6546048 ,
        -0.435114  ,  1.2100537 ,  0.2732106 ,  1.486608  , -1.0889524 ,
        -0.45181137,  0.80539733, -1.3142287 ,  1.6717596 , -1.0605558 ],
       dtype=float32),
 'many': array([-0.5303599 ,  0.34195098,  0.32928973,  0.401553  ,  0.07360838,
        -0.3130726 ,  1.3208677 ,  0.23534241,  0.22768378, -0.48265937,
         0.5584596 ,  0.4892218 , -0.21491633,  0.2641626 , -0.576728  ,
         1.5